#Dynamic router and Internet Tools:
Made by: Wilfredo Aaron Sosa Ramos (AI Lab Manager)

In [1]:
!pip install -q --upgrade langchain langchain-community langchain-core langchain-google-genai crewai

##1. Dynamic router:

In [2]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google GenAI API key: ")

Enter your Google GenAI API key: ··········


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
chat_google_genai = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [21]:
import time

def router(messages):
    start_time = time.time()
    messages_for_genai = [
        ("system", "You are an expert router. Use CoT (Chain of Thought) reasoning to evaluate and determine the appropriate action(s)."),
        ("human", f"""
          Evaluate the appropriate action(s) based on the given message:
          {messages[-1].content}

          Use CoT reasoning to determine if the task requires one or multiple actions. If it requires multiple actions, list all of them separated by commas. Use Few-shot prompting to cover edge cases and ambiguous requests.

          Choose from the following options:
          - 'translate': If the primary task is translating the text.
          - 'summarize': If the task requires creating a concise summary of the text.
          - 'rewrite': If the task involves rephrasing or improving the text while keeping its meaning.
          - 'question_generation': If the task is to generate questions based on the text.
          - 'custom': Use this for actions managing prompts (e.g., save prompt, update prompt, delete prompt).
          - 'default': If the task doesn’t fit any single category or involves other undefined operations.

          Examples:
          **Individual actions:**
          1. Input: "Translate this text into French." → Output: translate
          2. Input: "Summarize the following paragraph into a few sentences." → Output: summarize
          3. Input: "Rewrite this passage to make it more formal." → Output: rewrite
          4. Input: "Generate questions based on this content for a quiz." → Output: question_generation
          5. Input: "Save this new prompt to my library." → Output: custom
          6. Input: "Perform some operation on this text, but no specific instruction is provided." → Output: default

          **Complex tasks (multiple actions) - Don't return the CoT Reasoing, just the actions:**
          7. Input: "Translate this document into Spanish and then summarize the main points."
             CoT Reasoning: The task involves two sequential steps: translating the document into Spanish, followed by summarizing the translated content.
             Output: translate, summarize

          8. Input: "Rewrite this paragraph to improve clarity and generate three questions based on the revised version."
             CoT Reasoning: The task requires rephrasing the paragraph for clarity and then generating questions based on the revised text. Both actions are necessary.
             Output: rewrite, question_generation

          9. Input: "Summarize this article and translate the summary into German."
             CoT Reasoning: The task involves summarizing the article first and then translating the summary into German. Both actions are essential.
             Output: summarize, translate

          10. Input: "Generate questions from this essay and rewrite the questions in a simpler format."
              CoT Reasoning: The task involves generating questions from the essay and simplifying the generated questions through rewriting.
              Output: question_generation, rewrite

          11. Input: "Translate this text into French, summarize it, and rewrite the summary to make it more engaging."
              CoT Reasoning: The task involves three steps: translating into French, summarizing, and then rewriting the summary to improve engagement.
              Output: translate, summarize, rewrite

          Return only the name(s) of the selected action(s) in lowercase, separated by commas.
        """)
    ]

    result = chat_google_genai.invoke(messages_for_genai)
    end_time = time.time()  # Record the end time
    print(f"Request took: {end_time - start_time:.2f} seconds") # Print the elapsed time
    print(result.content.strip())
    return {"messages": messages, "action": result.content.strip(), "elapsed_time": round(end_time - start_time, 2)}


###Examples:

In [22]:
from langchain_core.messages import HumanMessage

input_message = "Texto a traducir"
target_language = "en"
source_language = "es"

messages = [
    HumanMessage(
        content=f"Please, translate '{input_message}' from {source_language} to {target_language}."
    )
]

router(messages)

Request took: 0.22 seconds
translate


{'messages': [HumanMessage(content="Please, translate 'Texto a traducir' from es to en.", additional_kwargs={}, response_metadata={})],
 'action': 'translate',
 'elapsed_time': 0.22}

In [23]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# Define 20 unique test cases in English
messages_list = [
    [HumanMessage(content="Could you convert 'This is a sample text' into Spanish?")],
    [HumanMessage(content="Summarize the following passage for a quick overview: 'This is a detailed paragraph about climate change.'")],
    [HumanMessage(content="Make this sentence sound more professional: 'Hey, can you fix this thing?'")],
    [HumanMessage(content="Formulate three questions from the text: 'The water cycle involves evaporation, condensation, and precipitation.'")],
    [HumanMessage(content="Store this template in my custom library for future use.")],
    [HumanMessage(content="Can you translate 'Here is a test document' into French and provide a summary?")],
    [HumanMessage(content="Generate quiz questions from 'Photosynthesis is how plants make food,' and simplify them for a young audience.")],
    [HumanMessage(content="Please translate 'This document needs to be in German,' summarize it, and rewrite it for clarity.")],
    [HumanMessage(content="What action should I take? No clear instructions were given for this text.")],
    [HumanMessage(content="Translate 'This is a complex paragraph' to Japanese and then create a set of discussion questions.")],
    [HumanMessage(content="Summarize this article and convert the summary into Italian: 'This is a long article about artificial intelligence.'")],
    [HumanMessage(content="Revise this passage for better readability and summarize the key points: 'Technical documents are often hard to read.'")],
    [HumanMessage(content="Add this text to my prompt collection: 'Rewrite this passage for simplicity.'")],
    [HumanMessage(content="Convert 'This text is about global warming' into Portuguese.")],
    [HumanMessage(content="Make this paragraph simpler and generate two questions about it: 'Quantum physics is a complex subject.'")],
    [HumanMessage(content="Change this paragraph to a more formal tone and then summarize it: 'Here is something casual.'")],
    [HumanMessage(content="No operation specified for this input: 'Just a random sentence.'")],
    [HumanMessage(content="Create a concise summary of this text and rewrite it to be more engaging: 'This is a technical report on robotics.'")],
    [HumanMessage(content="Translate 'A long document about machine learning' into English and write a short summary.")],
    [HumanMessage(content="Write two questions and their answers based on: 'Gravity keeps objects grounded on Earth.'")],
]

# Expected outputs for each test case
expected_outputs = [
    "translate",
    "summarize",
    "rewrite",
    "question_generation",
    "custom",
    "translate, summarize",
    "question_generation, rewrite",
    "translate, summarize, rewrite",
    "default",
    "translate, question_generation",
    "summarize, translate",
    "rewrite, summarize",
    "custom",
    "translate",
    "rewrite, question_generation",
    "rewrite, summarize",
    "default",
    "summarize, rewrite",
    "translate, summarize",
    "question_generation"
]

# Execute routing with multithreading
with ThreadPoolExecutor(max_workers=20) as executor:
    results = list(executor.map(router, messages_list))

# Save results in a DataFrame
df = pd.DataFrame([
    {
        "input_message": messages[0].content,
        "predicted_action": result["action"],
        "expected_action": expected_outputs[i],
        "elapsed_time (s)": result["elapsed_time"]
    }
    for i, (messages, result) in enumerate(zip(messages_list, results))
])

Request took: 0.25 seconds
question_generation
Request took: 0.27 seconds
translate
Request took: 0.26 seconds
question_generation, rewrite
Request took: 0.26 seconds
default
Request took: 0.28 seconds
custom
Request took: 0.26 seconds
question_generation
Request took: 0.29 seconds
summarize
Request took: 0.29 seconds
rewrite
Request took: 0.27 seconds
translate
Request took: 0.28 seconds
custom, rewrite
Request took: 0.29 seconds
translate, summarize, rewrite
Request took: 0.30 seconds
Request took: 0.28 seconds
rewrite, summarizetranslate, summarize

Request took: 0.28 seconds
summarize, rewriteRequest took: 0.30 seconds
default
Request took: 0.30 seconds
summarize, translate

Request took: 0.31 seconds
translate, question_generation
Request took: 0.31 seconds
rewrite, summarize
Request took: 0.38 seconds
rewrite, question_generation
Request took: 0.38 seconds
translate, summarize


In [24]:
df

,input_message,predicted_action,expected_action,elapsed_time (s)
0,Could you convert 'This is a sample text' into...,translate,translate,0.27
1,Summarize the following passage for a quick ov...,summarize,summarize,0.29
2,Make this sentence sound more professional: 'H...,rewrite,rewrite,0.29
3,Formulate three questions from the text: 'The ...,question_generation,question_generation,0.25
4,Store this template in my custom library for f...,custom,custom,0.28
5,Can you translate 'Here is a test document' in...,"translate, summarize","translate, summarize",0.30
6,Generate quiz questions from 'Photosynthesis i...,"question_generation, rewrite","question_generation, rewrite",0.26
7,Please translate 'This document needs to be in...,"translate, summarize, rewrite","translate, summarize, rewrite",0.29
8,What action should I take? No clear instructio...,default,default,0.30
9,Translate 'This is a complex paragraph' to Jap...,"translate, question_generation","translate, question_generation",0.31


In [30]:
messages_list2 = [
    [HumanMessage(content="Generate tips for differential instruction in a classroom setting.")],
    [HumanMessage(content="Suggest classroom management strategies for high school teachers.")],
    [HumanMessage(content="Provide feedback on this lesson plan: 'The lesson includes group activities and individual assessments.'")],
    [HumanMessage(content="Outline potential challenges in remote teaching and propose solutions.")],
    [HumanMessage(content="Develop methods to engage students in large classrooms.")],

    [HumanMessage(content="Translate 'This historical text' into German, summarize it, rewrite the summary for better clarity, and generate questions.")],
    [HumanMessage(content="Summarize the main points of this article, translate the summary to Spanish, rewrite it for clarity, and generate a quiz.")],
    [HumanMessage(content="Translate this technical document into French, summarize it, rewrite the summary for a beginner audience, and generate discussion questions.")],
    [HumanMessage(content="Translate this text into Japanese, summarize it, rewrite it for better engagement, and generate teaching materials based on it.")],
    [HumanMessage(content="Summarize the key points of this long document, translate the summary into three languages (Spanish, French, and German), and rewrite it for a professional audience.")],
]

expected_outputs2 = [
    # Expected outputs for new default examples
    "default",
    "default",
    "default",
    "default",
    "default",

    # Expected outputs for new complex prompts
    "translate, summarize, rewrite, question_generation",
    "summarize, translate, rewrite, question_generation",
    "translate, summarize, rewrite, question_generation",
    "translate, summarize, rewrite, question_generation",
    "summarize, translate, rewrite"
]

def run_simulation_for_router(messages_list, max_workers, expected_outputs):
  with ThreadPoolExecutor(max_workers=max_workers) as executor:
      results = list(executor.map(router, messages_list))

  df = pd.DataFrame([
      {
          "input_message": messages[0].content,
          "predicted_action": result["action"],
          "expected_action": expected_outputs[i],
          "elapsed_time (s)": result["elapsed_time"]
      }
      for i, (messages, result) in enumerate(zip(messages_list, results))
  ])

  return df

In [31]:
df2 = run_simulation_for_router(messages_list2, 10, expected_outputs2)

Request took: 0.24 seconds
default
Request took: 0.26 seconds
default
Request took: 0.27 seconds
default
Request took: 0.26 seconds
default
Request took: 0.28 seconds
default
Request took: 0.28 seconds
translate, summarize, rewrite, question_generation
Request took: 0.29 seconds
summarize, translate, rewrite, question_generation
Request took: 0.29 seconds
translate, summarize, rewrite, question_generation
Request took: 0.30 seconds
translate, summarize, rewrite, question_generation
Request took: 0.30 seconds
summarize, translate, rewrite


In [32]:
df2

,input_message,predicted_action,expected_action,elapsed_time (s)
0,Generate tips for differential instruction in ...,default,default,0.27
1,Suggest classroom management strategies for hi...,default,default,0.26
2,Provide feedback on this lesson plan: 'The les...,default,default,0.24
3,Outline potential challenges in remote teachin...,default,default,0.28
4,Develop methods to engage students in large cl...,default,default,0.26
5,"Translate 'This historical text' into German, ...","translate, summarize, rewrite, question_genera...","translate, summarize, rewrite, question_genera...",0.28
6,"Summarize the main points of this article, tra...","summarize, translate, rewrite, question_genera...","summarize, translate, rewrite, question_genera...",0.29
7,"Translate this technical document into French,...","translate, summarize, rewrite, question_genera...","translate, summarize, rewrite, question_genera...",0.29
8,"Translate this text into Japanese, summarize i...","translate, summarize, rewrite, question_genera...","translate, summarize, rewrite, question_genera...",0.30
9,Summarize the key points of this long document...,"summarize, translate, rewrite","summarize, translate, rewrite",0.30


##2. Internet Tools:

In [33]:
!mkdir config

In [76]:
%%writefile config/agents.yaml
resource_researcher:
  role: >
    Resource Researcher
  goal: >
    Gather high-quality and relevant resources (e.g., trusted websites, academic articles, news reports, videos, and blog posts) for a specified topic or query.
  backstory: >
    You are a seasoned researcher skilled at identifying credible sources and filtering out irrelevant or low-quality information. Your expertise lies in curating resources that meet high standards of accuracy and reliability.

content_verifier:
  role: >
    Content Verifier
  goal: >
    Verify the credibility, accuracy, and relevance of resources provided by the Resource Researcher.
  backstory: >
    You are a meticulous fact-checker with experience in analyzing the trustworthiness of online content. Your role is to ensure that only the most reliable and accurate resources are presented.

resource_presenter:
  role: >
    Resource Presenter
  goal: >
    Organize and format the verified resources into a clear and accessible list, ensuring that only real and functional URLs are included.
  backstory: >
    You specialize in presenting curated resources in a user-friendly manner. Your primary responsibility is to compile and deliver a list of verified URLs related to the topic or query.

url_curator:
  role: >
    URL Curator
  goal: >
    Ensure that the final output is a refined list of real and functional URLs, without additional analysis or summaries.
  backstory: >
    You are a precision-oriented curator focused exclusively on presenting a simple, accurate list of real, working URLs retrieved for a given topic or query.

Overwriting config/agents.yaml


In [35]:
%%writefile config/tasks.yaml
resource_researcher_task:
  description: >
    Conduct a comprehensive search for high-quality and relevant resources (e.g., trusted websites, academic articles, news reports, videos, and blog posts) on the topic: {query_topic}.
    Focus on gathering real, recent, and reliable information.
  expected_output: >
    A list of at least 10 URLs to credible sources related to {query_topic}, including a brief annotation (1-2 lines) describing the relevance of each source.

content_verifier_task:
  description: >
    Verify the credibility, accuracy, and functionality of the URLs gathered by the Resource Researcher for the topic: {query_topic}.
    Ensure that all URLs are from trusted sources and provide useful information.
  expected_output: >
    A refined list of verified URLs for {query_topic}, free of any non-functional or irrelevant links.

resource_presenter_task:
  description: >
    Organize and format the list of verified URLs into a clear, accessible format.
    Ensure that the URLs are real and functional and avoid additional annotations or analysis.
  expected_output: >
    A well-formatted list of functional URLs related to {query_topic}, ready for presentation to the requester.

url_curator_task:
  description: >
    Focus on presenting the final list of URLs in a precise, easy-to-read manner.
    Do not include summaries or descriptions—just the URLs for {query_topic}.
  expected_output: >
    A simple and accurate list of curated URLs for {query_topic}.

Writing config/tasks.yaml


In [36]:
!pip install -q crewai_tools

In [95]:
from crewai.tools import tool
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

@tool
def tavily_search(query: str) -> str:
    """Perform a search with Tavily, including answers and raw content."""

    tavily_search_wrapper = TavilySearchAPIWrapper()

    return tavily_search_wrapper.results(query=query, max_results=5, search_depth="advanced")

In [38]:
import os
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")
os.environ["SERPER_API_KEY"] = getpass.getpass("Enter your Serper API key: ")

Enter your Tavily API key: ··········
Enter your Serper API key: ··········


In [45]:
!pip install --upgrade crewai crewai-tools crewai[tools]

In [49]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from typing import List
from pydantic import BaseModel, Field

In [50]:
# Define structured output for tasks
class URLResource(BaseModel):
    url: str = Field(..., description="The URL of the resource")
    annotation: str = Field(..., description="Brief description of the URL's relevance")

In [80]:
import yaml

with open("config/agents.yaml", "r") as f:
    agent_config = yaml.safe_load(f)

###Agents:

In [103]:
os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [107]:
from crewai import LLM
llm_openai = LLM(model="gpt-4o-mini")

In [81]:
agent_config

{'resource_researcher': {'role': 'Resource Researcher\n',
  'goal': 'Gather high-quality and relevant resources (e.g., trusted websites, academic articles, news reports, videos, and blog posts) for a specified topic or query.\n',
  'backstory': 'You are a seasoned researcher skilled at identifying credible sources and filtering out irrelevant or low-quality information. Your expertise lies in curating resources that meet high standards of accuracy and reliability.\n'},
 'content_verifier': {'role': 'Content Verifier\n',
  'goal': 'Verify the credibility, accuracy, and relevance of resources provided by the Resource Researcher.\n',
  'backstory': 'You are a meticulous fact-checker with experience in analyzing the trustworthiness of online content. Your role is to ensure that only the most reliable and accurate resources are presented.\n'},
 'resource_presenter': {'role': 'Resource Presenter\n',
  'goal': 'Organize and format the verified resources into a clear and accessible list, ensur

In [56]:
resource_researcher_config = agent_config["resource_researcher"]

In [108]:
from crewai import LLM

resource_researcher = Agent(
    role=resource_researcher_config["role"],
    goal=resource_researcher_config["goal"],
    backstory=resource_researcher_config["backstory"],
    tools=[SerperDevTool(), ScrapeWebsiteTool(), tavily_search],
    llm=llm_openai,
    verbose=True,
    memory=False,
)

In [110]:
# Content Verifier
content_verifier_config = agent_config["content_verifier"]
content_verifier = Agent(
    role=content_verifier_config["role"],
    goal=content_verifier_config["goal"],
    backstory=content_verifier_config["backstory"],
    tools=[SerperDevTool(), tavily_search],
    llm=llm_openai,
    verbose=True,
    memory=False,
)

# Resource Presenter
resource_presenter_config = agent_config["resource_presenter"]
resource_presenter = Agent(
    role=resource_presenter_config["role"],
    goal=resource_presenter_config["goal"],
    backstory=resource_presenter_config["backstory"],
    llm=llm_openai,
    verbose=True,
    memory=False,
)

# URL Curator
url_curator_config = agent_config["url_curator"]
url_curator = Agent(
    role=url_curator_config["role"],
    goal=url_curator_config["goal"],
    backstory=url_curator_config["backstory"],
    llm=llm_openai,
    verbose=True,
    memory=False,
)

###Tasks:

In [72]:
with open("config/tasks.yaml", "r") as f:
    task_config = yaml.safe_load(f)

In [73]:
task_config

{'resource_researcher_task': {'description': 'Conduct a comprehensive search for high-quality and relevant resources (e.g., trusted websites, academic articles, news reports, videos, and blog posts) on the topic: {query_topic}. Focus on gathering real, recent, and reliable information.\n',
  'expected_output': 'A list of at least 10 URLs to credible sources related to {query_topic}, including a brief annotation (1-2 lines) describing the relevance of each source.\n'},
 'content_verifier_task': {'description': 'Verify the credibility, accuracy, and functionality of the URLs gathered by the Resource Researcher for the topic: {query_topic}. Ensure that all URLs are from trusted sources and provide useful information.\n',
  'expected_output': 'A refined list of verified URLs for {query_topic}, free of any non-functional or irrelevant links.\n'},
 'resource_presenter_task': {'description': 'Organize and format the list of verified URLs into a clear, accessible format. Ensure that the URLs a

In [75]:
from textwrap import dedent
from crewai import Process

# Resource Researcher Task
resource_researcher_task = Task(
    description=dedent(task_config["resource_researcher_task"]["description"]),
    agent=resource_researcher,
    expected_output=task_config["resource_researcher_task"]["expected_output"],
)

# Content Verifier Task
content_verifier_task = Task(
    description=dedent(task_config["content_verifier_task"]["description"]),
    agent=content_verifier,
    context=[resource_researcher_task],
    expected_output=task_config["content_verifier_task"]["expected_output"],
)

# Resource Presenter Task
resource_presenter_task = Task(
    description=dedent(task_config["resource_presenter_task"]["description"]),
    agent=resource_presenter,
    context=[content_verifier_task],
    expected_output=task_config["resource_presenter_task"]["expected_output"],
)

# URL Curator Task
url_curator_task = Task(
    description=dedent(task_config["url_curator_task"]["description"]),
    agent=url_curator,
    context=[resource_presenter_task],
    expected_output=task_config["url_curator_task"]["expected_output"],
)

###Crew:

In [111]:
resource_search_crew = Crew(
    agents=[resource_researcher, content_verifier, resource_presenter, url_curator],
    tasks=[resource_researcher_task, content_verifier_task, resource_presenter_task, url_curator_task],
    process=Process.sequential,
    verbose=True,
)

In [112]:
if __name__ == "__main__":
    inputs = {"query_topic": "Latest advancements in AI research"}
    result = resource_search_crew.kickoff(inputs=inputs)
    print(result)

# Agent: Resource Researcher
## Task: Conduct a comprehensive search for high-quality and relevant resources (e.g., trusted websites, academic articles, news reports, videos, and blog posts) on the topic: Latest advancements in AI research. Focus on gathering real, recent, and reliable information.



# Agent: Resource Researcher
## Thought: Thought:I need to use a search engine to find relevant resources on the latest advancements in AI research.  I'll focus my search on reputable sources like academic journals, news outlets known for their science reporting, and well-respected technology publications.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest advancements in AI research\"}"
## Tool Output: 

Search results: Title: 7 Recent AI Developments: Artificial Intelligence News - Koombea
Link: https://www.koombea.com/blog/7-recent-ai-developments/
Snippet: AI Robots Learning Through Observation · AI Robot Caregivers Are Filling a Shortfall · AI Beer Brewer

In [113]:
result

CrewOutput(raw='* https://www.sciencedaily.com/news/computers_math/artificial_intelligence/\n* https://www.technologyreview.com/2024/01/08/1085096/artificial-intelligence-generative-ai-chatgpt-open-ai-breakthrough-technologies/\n* https://arxiv.org/list/cs.AI/current\n* https://aiindex.stanford.edu/report/\n* https://www.ibm.com/think/insights/artificial-intelligence-trends\n* https://www.gao.gov/blog/artificial-intelligences-use-and-rapid-growth-highlight-its-possibilities-and-perils\n* https://ai.meta.com/results/?page=1&content_types[0]=publication', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Conduct a comprehensive search for high-quality and relevant resources (e.g., trusted websites, academic articles, news reports, videos, and blog posts) on the topic: Latest advancements in AI research. Focus on gathering real, recent, and reliable information.\n', name=None, expected_output='A list of at least 10 URLs to credible sources related to Latest advancements